Añadir tablas de DEpartment for Transportation (UK) para informacion en puertos:

In [2]:
import pandas as pd
import xlrd
import re
from curating_functions import remove_note

**Port0602**: counts of vessels - cargo and passenger - arriving at UK major ports, and selected other ports, from 2009 

In [3]:
ficheros_uk = '../home_uk/port0602.xls' # Otras fuentes: 
uk_number_vessels_ports = pd.read_excel(ficheros_uk,
                                   sheet_name ='port0602',
                                   header=8,
                                   usecols="A:Q",
                                   skiprows=[9,122,123,124,125,126,127,128,129],
                                   engine='xlrd'
                                   )

Entre 2017 y 2017.1 hay un cambio en la contabilidad

**Port0601**: counts of vessels - cargo and passenger - arriving at UK major ports, and selected other ports, from 2009 **by vessel type and size**\. $\red{Rechazada~por~falta~de~granularidad}$
**Port0302**: UK major port freight traffic by port and route (filter by direction, cargo group and year):
    - $\orange{Aporta~informacion~sobre~tipo~de~barco~entrante~o~saliente~con~cargamento~y~puerto.~PUEDE~SERVIR}$\
    - port0302: Table with Port, freight by scope\
    - Data: Year | Port | Cargo Group | REgion | Direction | Tonnage | Units



**Port0301**: UK major port freight traffic by port and year (filter by direction and cargo type), from 2000\
Metric: Tonnage

In [4]:
ficheros_uk = '../home_uk/port0301.xls' # Otras fuentes: 
uk_ports_freight = pd.read_excel(ficheros_uk,
                                   sheet_name ='port0301', # ver hoja 'Data'
                                   header=10,
                                   usecols="A:Q",
                                   skiprows=[40,46,59,67,68],
                                   engine='xlrd'
                                   )

**port0203**: UK major port unitised main freight traffic by port, cargo type, metric and loaded status (filter by direction and year)\
**fle0100**: UK Ship Register annual statistics and commercial data comparison: 2003 to 2023 (No desagregado)

Para Port0302:
 - La hoja 'port0302' aporta información sobre tonelaje de cargamento para cada puerto segun domestic/international...
 - La hoja 'Data' especifica cada grupo de 'Cargo' por puerto y año y el sentido (entrada/salida). $\green{Usaremos~estos~datos}$

In [5]:
# descargar Port0302:
ficheros_uk = '../home_uk/port0302.xls' # Otras fuentes: 
uk_vessels_freight = pd.read_excel(ficheros_uk,
                                   sheet_name ='Data', # ver hoja 'Data'
                                   header=0,
                                   usecols="A:J",
                                   engine='xlrd'
                                   )

Incluir los ficheros extraidos del zip:
- **port0502**: UK major port traffic by port (filter by direction), quarterly from 2009 (metric: thousand tonnes (tonnage). $\green{Puede~servir}$)
- **port0503**: UK major port traffic by port (filter by direction and cargo group). $\orange{No~se~perciben~muchas~diferencias~respecto~a~port0502}$

In [111]:
ficheros_uk = '../home_uk/quarterly-port-freight-statistics-july-to-september-2024-tables/port0502.xls' # Otras fuentes: 
uk_port_freight_quaterly = pd.read_excel(ficheros_uk,
                                   sheet_name ='PORT0502', # ver hoja 'Data'
                                   header=9,
                                   usecols="A:BL",
                                   skiprows=[63],
                                   engine='xlrd'
                                   )

### Union de dataframes con Puertos:

1. Limpiar columnas con [Notes]

In [7]:
uk_ports_freight['Port'] = uk_ports_freight['Port'].apply(remove_note)

In [8]:
uk_number_vessels_ports[' Port'] = uk_number_vessels_ports[' Port'].apply(remove_note)
uk_number_vessels_ports.rename(columns={' Port':'Port'}, inplace=True)

In [9]:
uk_vessels_freight['Port'] = uk_vessels_freight['Port'].apply(remove_note)

In [112]:
uk_port_freight_quaterly.columns

Index(['Major Port', '2009 Q1', '2009 Q2', '2009 Q3', '2009 Q4', '2010 Q1',
       '2010 Q2', '2010 Q3', '2010 Q4', '2011 Q1', '2011 Q2', '2011 Q3',
       '2011 Q4', '2012 Q1', '2012 Q2', '2012 Q3', '2012 Q4', '2013 Q1',
       '2013 Q2', '2013 Q3', '2013 Q4', '2014 Q1', '2014 Q2', '2014 Q3',
       '2014 Q4', '2015 Q1', '2015 Q2', '2015 Q3', '2015 Q4', '2016 Q1',
       '2016 Q2', '2016 Q3', '2016 Q4', '2017 Q1', '2017 Q2', '2017 Q3',
       '2017 Q4', '2018 Q1', '2018 Q2', '2018 Q3', '2018 Q4', '2019 Q1',
       '2019 Q2', '2019 Q3', '2019 Q4', '2020 Q1', '2020 Q2\n [Note 9]',
       '2020 Q3', '2020 Q4', '2021 Q1', '2021 Q2', '2021 Q3', '2021 Q4',
       '2022 Q1', '2022 Q2', '2022 Q3', '2022 Q4', '2023 Q1', '2023 Q2',
       '2023 Q3', '2023 Q4', '2024 Q1 [Note 10]', '2024 Q2 [Note 10]',
       '2024 Q3 [Note 10]'],
      dtype='object')

In [113]:
uk_port_freight_quaterly['Major Port'] = uk_port_freight_quaterly['Major Port'].apply(remove_note)
uk_port_freight_quaterly.at[27, "Major Port"] = uk_port_freight_quaterly.at[27, "Major Port"].split(" [")[0]

In [10]:
df1 = uk_number_vessels_ports[["Port","Major/Minor ports","Port group"]]
df2 = uk_ports_freight[["Port","UK Country"]]

df_ports_uk = pd.merge(left = df1,
         right = df2,
         how = 'outer',
         left_on=['Port', df1.groupby('Port').cumcount()],
         right_on=['Port', df2.groupby('Port').cumcount()])

df_ports_uk['id_Port'] = df_ports_uk.index

In [61]:
puertos_uk = df_ports_uk["Port"]
puertos_uk = puertos_uk.tolist()

In [ ]:
unique_ports = uk_vessels_freight["Port"].unique()
unique_ports_list = unique_ports.tolist()

In [74]:
n = 0
for i in unique_ports_list:
    if i in puertos_uk:
        n += 1
    else:
        print(i)
        #Add i to df_ports_uk
        new_row = pd.DataFrame({"Port":[i],"id_Port":[len(df_ports_uk)]})
        df_ports_uk = pd.concat([df_ports_uk, new_row], ignore_index=False)

print(n)

Loch Ryan
Southampton
Stranraer
50


In [127]:
df_ports_uk.loc[df_ports_uk["Port"] == "Grimsby and Immingham"] 

,Port,key_1,Major/Minor ports,Port group,UK Country,id_Port
0,Grimsby and Immingham,NaN,NaN,NaN,NaN,124


In [114]:
unique_ports = uk_port_freight_quaterly["Major Port"].unique()
unique_ports_list = unique_ports.tolist()

In [116]:
n = 0
for i in unique_ports_list:
    if i in puertos_uk:
        n += 1
    else:
        print(i)
        #Add i to df_ports_uk
        new_row = pd.DataFrame({"Port":[i],"id_Port":[len(df_ports_uk)]})
        df_ports_uk = pd.concat([df_ports_uk, new_row], ignore_index=False)

print(n)

Cromarty Firth 
Goole 
Grimsby and Immingham
Loch Ryan
Poole 
Rivers Hull and Humber 
Southampton
Sullom Voe 
45


Se ha creado la tabla/dataframe "UK_PORTS" con el identificador (PK) id_Port, detrayendo de los dataframes originales las columnas que caracterizan a los puertos.
Ahora, de esos primeros dataframes, una vez extraidas las columnas de atributos de puerto, se sustituye el nombre de puerto por el numero de identificador (que hace las veces de foreign key para la tabla "UK PORTS")

In [11]:
key_value_dict = dict(zip(df_ports_uk['Port'], df_ports_uk['id_Port'])) 

In [20]:
uk_freight = uk_ports_freight.iloc[:, [1] + list(range(3, len(uk_ports_freight.columns)-1))]
uk_freight.rename(columns={'Port':'FK_Port'}, inplace=True)
uk_freight['id_Port'] = uk_freight['FK_Port'].map(key_value_dict)

/tmp/ipykernel_38638/2293702691.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uk_freight.rename(columns={'Port':'FK_Port'}, inplace=True)
/tmp/ipykernel_38638/2293702691.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uk_freight['id_Port'] = uk_freight['FK_Port'].map(key_value_dict)


Se ha incluido la columna id_Port que es la foreign key que la relaciona con Ports_UK:

Se hace lo mismo para UK_vessels:

In [21]:
uk_vessels_in_port = uk_number_vessels_ports.iloc[:, [0] + list(range(3, len(uk_ports_freight.columns)-1))]
uk_vessels_in_port['id_Port'] = uk_vessels_in_port['Port'].map(key_value_dict)

/tmp/ipykernel_38638/32495670.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uk_vessels_in_port['id_Port'] = uk_vessels_in_port['Port'].map(key_value_dict)


In [81]:
uk_vessels_freight = uk_vessels_freight.drop(uk_vessels_freight.columns[2], axis=1)
uk_vessels_freight['id_Port'] = uk_vessels_freight['Port'].map(key_value_dict)

In [129]:
uk_port_freight_quaterly['id_Port'] = uk_vessels_freight['Major Port'].map(key_value_dict)

Eliminar duplicados de df_ports_uk

Por ejemplo, London solo sale en la tabla 'freight' pero no en 'vessels_in_port'. Es posible que haya que considerar entidades regionales que engloben distintos puertos (ver la columna 'Port group' de df_ports_uk)

Ya tenemos las tablas df_ports_uk, uk_freight y uk_vessels_in_port, que relacionan puertos, regiones, con numero de barcos y volumen de cargamento.\
Con la cantidad de tonelaje que entra en cada puerto (o sale) se puede identificar si el puerto es un puerto de carga (de containers), pesquero, deportivo o de otro tipo.



*Guardar .csv de los dataframes creados:

In [168]:
uk_vessels_in_port.to_csv('../home_uk/uk_vessels_in_port.csv', index=False)
uk_freight.to_csv('../home_uk/uk_freight.csv', index=False)
uk_vessels_freight.to_csv('../home_uk/uk_freight_in_vessels.csv', index=False)
uk_port_freight_quaterly.to_csv('../home_uk/uk_port_freight_quaterly.csv', index=False)

In [138]:
df_ports_uk["Port"].unique().tolist()

['Aberdeen',
 'Appledore',
 'Ardrishaig',
 'Ayr',
 'Barrow',
 'Barry',
 'Belfast',
 'Berwick-On-Tweed',
 'Bideford',
 'Blyth',
 'Boston',
 'Bridgwater',
 'Brightlingsea',
 'Bristol',
 'Buckie',
 'Burry Port',
 'Cairnryan',
 'Cardiff',
 'Chichester',
 'Clyde',
 'Coleraine',
 'Corpach',
 'Cowes, Isle of Wight ',
 'Cromarty Firth',
 'Dover',
 'Dundee',
 'Falmouth',
 'Felixstowe',
 'Fishguard',
 'Fleetwood',
 'Forth',
 'Fowey',
 'Fraserburgh',
 'Garston',
 "Gill's Bay Scotland",
 'Glensanda',
 'Goole',
 'Great Yarmouth',
 'Grimsby & Immingham',
 'Harwich',
 'Heysham',
 'Holyhead',
 "Hughtown (St Mary's)",
 'Hull',
 'Inverkeithing',
 'Inverness',
 'Ipswich',
 'Kilroot',
 'Kilroot Power Station Jetty',
 "King's Lynn",
 'Lancaster',
 'Larne',
 'Lerwick',
 'Littlehampton',
 'Liverpool',
 'Llandulas',
 'Loch Ryan ',
 'London',
 'Londonderry',
 'Lowestoft',
 'Manchester',
 'Medway',
 'Milford Haven',
 'Mistley',
 'Montrose',
 'Mostyn',
 'Neath',
 'Newhaven',
 'Newport',
 'Newport, Isle of Wight'

In [141]:
df_ports_uk["Port"] = df_ports_uk["Port"].str.rstrip()
df_ports_uk.drop_duplicates(subset=['Port'], keep='first', inplace=True)

### Cargar CSV drug_seizure UK
Vamos a relacionar el puerto con la región o autoridad pertinente

In [149]:
uk_drug_seizure_df = pd.read_csv('../home_uk/uk_drug_seizure_df.csv',
            sep = ',',
            header = 0,
            skiprows = [44,45,46,47])

Ver: https://medium.com/@alexroz/6-python-libraries-to-make-beautiful-maps-9fb9edb28b27
- cartopy
- folium and geopandas
- plotly
- ipyleaflet
- geemap
- ridgemap

Para vincular el puerto con el Area Policial, chatGPT recomienda:
1. Si se tienen las coordenadas de los puertos y de las areas policiales (AP): geopandas + spatial join
2. Si se tiene informacion sobre la ciudad o CP: uso de APIs publicas,

En este caso, se prueba primero con [2]

*Otra opcion es mediante 'embeddings' si diera tiempo.

In [177]:
# Primera solucion aportada por ChatGPT
port_group_to_police = {
    'Scotland East Coast': 'Northumbria',
    'Bristol Channel': 'Avon & Somerset',
    'Scotland West Coast': 'Cumbria',
    'Lancs and Cumbria': 'Cumbria',
    'Northern Ireland': 'Northern Ireland',  # Ajusta según necesidad
    'North East': 'Cleveland',
    'Wash & Northern E Anglia': 'Norfolk ',
    'Thames and Kent': 'Kent ',
    'Sussex and Hampshire': 'Sussex',
    'West Country': 'Devon & Cornwall ',
    'Haven': 'Essex',
    'West and North Wales': 'North Wales',
    'Humber': 'Humberside'
}

# Asignar la región policial al dataframe de puertos
df_ports_uk['Police force region'] = df_ports_uk['Port group'].map(port_group_to_police)

In [179]:
df_ports_uk.to_csv('../home_uk/uk_ports.csv', index=False)